In [ ]:
pi = RR.pi()

In [ ]:
def scale(a, v):
    return vector([a * x for x in v])

In [ ]:
X(u, v) = (sin(v) * cos(u), sin(v) * sin(u), cos(v))
parametric_plot3d(X(u, v), (u, -pi, pi), (v, 0, pi), frame=False,axes=False,color='purple',opacity=1)

In [ ]:
a = 5
b = 1
T2(theta, phi) = ((a + b * cos(theta)) * cos(phi), (a + b * cos(theta)) * sin(phi), b * sin(theta))
parametric_plot3d(T2(theta, phi), (theta, 0, 2 * pi), (phi, 0, 2 * pi), frame=False,color='purple',opacity=1)

In [ ]:
torus_plot = parametric_plot3d(T2(theta, phi), (theta, 0, 2 * pi), (phi, 0, 2 * pi), frame=False,color='purple',opacity=0.5)
curve_plot = parametric_plot3d(T2(theta, (2/3) * theta), (theta, 0, 1000), thickness=3, plot_points=10000)
show(torus_plot + curve_plot)
# show(curve_plot)

In [ ]:
def compute_FFF(X_u, X_v):
    return (X_u.inner_product(X_u), X_u.inner_product(X_v), X_v.inner_product(X_v))

In [ ]:
def compute_christoffel(X_1, X_2, u, v, E_, F_, G_):
    E_u = diff(E_, X_1).subs(X_1 == u, X_2 == v)
    E_v = diff(E_, X_2).subs(X_1 == u, X_2 == v)
    F_u = diff(F_, X_1).subs(X_1 == u, X_2 == v)
    F_v = diff(F_, X_2).subs(X_1 == u, X_2 == v)
    G_u = diff(G_, X_1).subs(X_1 == u, X_2 == v)
    G_v = diff(G_, X_2).subs(X_1 == u, X_2 == v)

    E = E_.subs(X_1 == u, X_2 == v)
    F = F_.subs(X_1 == u, X_2 == v)
    G = G_.subs(X_1 == u, X_2 == v)
    
    C111 = var('C111')
    C121 = var('C121')
    C221 = var('C221')
    C112 = var('C112')
    C122 = var('C122')
    C222 = var('C222')
    
    eqn_11 = C111 * E + C112 * F == 1/2 * E_u
    eqn_12 = C111 * F + C112 * G == F_u - 1/2 * E_v
    
    eqn_21 = C121 * E + C122 * F == 1/2 * E_v
    eqn_22 = C121 * F + C122 * G == 1/2 * G_u
    
    eqn_31 = C221 * E + C222 * F == F_v - 1/2 * G_u
    eqn_32 = C221 * F + C222 * G == 1/2 * G_v
    
    C_sol_1 = solve([eqn_11, eqn_12], C111, C112)
    C_sol_2 = solve([eqn_21, eqn_22], C121, C122)
    C_sol_3 = solve([eqn_31, eqn_32], C221, C222)
    
    C111 = C111.subs(C_sol_1[0])
    C112 = C112.subs(C_sol_1[0])
    C121 = C121.subs(C_sol_2[0])
    C122 = C122.subs(C_sol_2[0])
    C221 = C221.subs(C_sol_3[0])
    C222 = C222.subs(C_sol_3[0])

    return (C111, C112, C121, C122, C221, C222)

In [ ]:
def PT(X, X_1, X_2, u, v, T, initial_vec=None, step=0.1, initial_vec_length=None):
    t = T[0]
    t_0 = T[1]
    t_f = T[2]

    X_u = diff(X, X_1)
    X_v = diff(X, X_2)
    
    E, F, G = compute_FFF(X_u, X_v)
    
    C111, C112, C121, C122, C221, C222 = compute_christoffel(X_1, X_2, u, v, E, F, G)
    
    a = var('a')
    b = var('b')
    
    v_t = diff(v, t)
    u_t = diff(u, t)
    
    diffeq_a = -(C111 * u_t + C121 * v_t) * a - (C121 * u_t + C221 * v_t) * b
    diffeq_b = -(C112 * u_t + C122 * v_t) * a - (C122 * u_t + C222 * v_t) * b
    
    if initial_vec == None:
        initial_vec = (u_t(t_0), v_t(t_0))

    if initial_vec_length is not None:
        initial_vec_on_surface = X_u(u(t_0), v(t_0)) * initial_vec[0] + X_v(u(t_0), v(t_0)) * initial_vec[1]
        length = initial_vec_on_surface.norm()
        initial_vec = ((initial_vec[0] / length) * initial_vec_length, (initial_vec[1] / length) * initial_vec_length)

    P = desolve_system_rk4([diffeq_a, diffeq_b], [a, b], ics=[t_0,initial_vec[0],initial_vec[1]], ivar=t, end_points=t_f, step=step)
    return P

In [ ]:
def sphere_PT(meridian=pi/2):
    t = var('t')
    phi,theta = var('φ', 'θ')

    u(t) = t
    v(t) = meridian

    X(theta, phi) = (cos(theta) * cos(phi), sin(theta) * cos(phi), sin(phi))
    X_theta = diff(X, theta)
    X_phi = diff(X, phi)

    P = PT(X, theta, phi, u, v, (t, 0, 2 * pi))

    X_plot = parametric_plot3d(X(theta, phi), (theta, 0, 2 * pi), (phi, 0, 2 * pi), frame=False,axes=True,color='purple',opacity=1)
    C_plot = parametric_plot3d(X(u(t), v(t)), (t, 0, 2 * pi), frame=False, Axes=False,color='blue',opacity=1)

    num_points = len(P)
    num_to_plot = len(P)
    period = int(num_points / num_to_plot)
    vectors = []
    for ti,a,b in P[::period]:
        vec = (scale(a, X_theta(u(ti), v(ti))) + scale(b, X_phi(u(ti), v(ti))))
        vectors.append(arrow(X(u(ti), v(ti)), X(u(ti), v(ti)) + vec))

    output = X_plot + sum(vectors) + C_plot
    output.show()

In [ ]:
sphere_PT(meridian=0)

In [ ]:
sphere_PT(meridian=pi/5)

In [ ]:
sphere_PT(meridian=pi/4)

In [ ]:
sphere_PT(meridian=pi/3)

In [ ]:
def class_logo_PT(angle=pi/2, meridian=pi/2, num_to_plot=10):
    t = var('t')
    phi,theta = var('φ', 'θ')
    X(theta, phi) = (sin(phi) * cos(theta), sin(phi) * sin(theta), cos(phi))
    X_theta = diff(X, theta)
    X_phi = diff(X, phi)

    epsilon = 0.01
    step=0.01

    u_1(t) = 0
    v_1(t) = t

    u_2(t) = t
    v_2(t) = meridian

    u_3(t) = angle
    v_3(t) = t

    print("PT 1")
    PT_1 = PT(X, theta, phi, u_1, v_1, (t, epsilon, meridian), (1, 0), step=step, initial_vec_length=0.5)
    final_PT_1 = PT_1[-1]

    print("PT 2")
    PT_2 = PT(X, theta, phi, u_2, v_2, (t, 0, angle), (final_PT_1[1], final_PT_1[2]), step)
    final_PT_2 = PT_2[-1]

    print("PT 3")
    PT_3 = PT(X, theta, phi, u_3, v_3, (t, meridian, 0), (final_PT_2[1], final_PT_2[2]), step)

    X_plot = parametric_plot3d(X(theta, phi), (theta, 0, 2 * pi), (phi, 0, pi), frame=False,axes=False,color='purple',opacity=1)
    vectors = []

    count = -1
    num_points = len(PT_1 + PT_2 + PT_3)

    for ti,a,b in PT_1[::int(len(PT_1) / num_to_plot)]:
        vec = (scale(a, X_theta(u_1(ti), v_1(ti))) + scale(b, X_phi(u_1(ti), v_1(ti))))
        vectors.append(arrow(X(u_1(ti), v_1(ti)), X(u_1(ti), v_1(ti)) + vec))

    for ti,a,b in PT_2[::int(len(PT_2) / num_to_plot)]:
        vec = (scale(a, X_theta(u_2(ti), v_2(ti))) + scale(b, X_phi(u_2(ti), v_2(ti))))
        vectors.append(arrow(X(u_2(ti), v_2(ti)), X(u_2(ti), v_2(ti)) + vec))

    for ti,a,b in reversed(PT_3[::int(len(PT_3) / num_to_plot)]):
        vec = (scale(a, X_theta(u_3(ti), v_3(ti))) + scale(b, X_phi(u_3(ti), v_3(ti))))
        vectors.append(arrow(X(u_3(ti), v_3(ti)), X(u_3(ti), v_3(ti)) + vec))

    C_1_plot = parametric_plot3d(X(u_1(t), v_1(t)), (t, 0, meridian), frame=False, axes=False, color='blue', opacity=1)
    C_2_plot = parametric_plot3d(X(u_2(t), v_2(t)), (t, 0, angle), frame=False, axes=False, color='blue', opacity=1)
    C_3_plot = parametric_plot3d(X(u_3(t), v_3(t)), (t, 0, meridian), frame=False, axes=False, color='blue', opacity=1)

    show(X_plot + C_1_plot + C_2_plot + C_3_plot + sum(vectors))

In [ ]:
class_logo_PT(angle=pi/2, meridian=pi/2)

In [ ]:
class_logo_PT(angle=pi/2, meridian=pi/3)

In [ ]:
class_logo_PT(angle=pi/2, meridian=pi/4)

In [ ]:
class_logo_PT(angle=pi/2, meridian=3*pi/2, num_to_plot=20)

In [ ]:
class_logo_PT(angle=5*pi/6, meridian=3*pi/4)

In [ ]:
import numpy as np

t = var('t')
u = var('u')
v = var('v')
c = var('c')

X(u, v, c) = (u, v, 1/c * log(cos(v)/cos(u)))
def make_plot(c, urange=(-pi,pi), vrange=(-pi,pi)):
    return parametric_plot3d(X(u, v, c), (u, urange[0], urange[1]), (v, vrange[0], vrange[1]), plot_points=[200, 200])

In [ ]:
make_plot(1, (-pi, pi), (-pi, pi)).show()

In [ ]:
P = make_plot(1, (-pi/2, pi/2), (-pi/2, pi/2))

# If no condition is applied, plot looks bad because z value goes to infinity
def condition(x, y, z):
    return bool(-2 < z < 2)

R = P.add_condition(condition)
R.show()


In [ ]:
make_plot(1, (-4*pi, 4*pi), (-4*pi, 4*pi))

In [ ]:
P = make_plot(1, (-4*pi, 4*pi), (-4*pi, 4*pi))
def condition(x, y, z):
    return bool(-2 < z < 2)

R = P.add_condition(condition)
R.show()

In [ ]:
make_plot(1, (-pi, pi), (-pi, pi)).show()

In [ ]:
make_plot(2, (-pi, pi), (-pi, pi)).show()

In [ ]:
make_plot(3, (-pi, pi), (-pi, pi)).show()

In [ ]:
make_plot(4, (-pi, pi), (-pi, pi)).show()

In [ ]:
make_plot(10, (-pi, pi), (-pi, pi)).show()

In [ ]:
def transformation_plot(X, theta, urange=(-pi, pi), vrange=(-1, 1)):
    return ParametrizedSurface3D(X(u, v, theta), (u, v)).plot(urange, vrange, opacity=0.9)


def lines_of_curvature_plots(X, theta, urange=(-pi, pi, 20), vrange=(-1, 1, 20), color='black', thickness=2):
    lines = []
    
    for ui in np.linspace(urange[0], urange[1], urange[2]):
        v = var('v')
        lines.append(parametric_plot3d(X(ui, v, theta), (v, vrange[0], vrange[1]), color=color, thickness=thickness))
    for vi in np.linspace(vrange[0], vrange[1], vrange[2]):
        u = var('u')
        lines.append(parametric_plot3d(X(u, vi, theta), (u, urange[0], urange[1]), color=color, thickness=thickness))
    return lines


def get_total_plot(X, theta):
    return transformation_plot(X, theta) \
        + parametric_plot3d(X(u, 0, theta), (u, -pi, pi), color='red', thickness=2) \
        + sum(lines_of_curvature_plots(X, theta))


u = var('u')
v = var('v')

x(u, v, theta) = cos(theta) * sinh(v) * sin(u) + sin(theta) * cosh(v) * cos(u)
y(u, v, theta) = -cos(theta) * sinh(v) * cos(u) + sin(theta) * cosh(v) * sin(u)
z(u, v, theta) = u * cos(theta) + v * sin(theta)
X(u, v, theta) = (x(u, v, theta), y(u, v, theta), z(u, v, theta))

In [ ]:
get_total_plot(X, 0).show()

In [ ]:
get_total_plot(X, pi/5).show()

In [ ]:
get_total_plot(X, pi/4).show()

In [ ]:
get_total_plot(X, pi/3).show()

In [ ]:
get_total_plot(X, pi/2).show()

In [ ]:
enneper = surfaces.Enneper()
enneper.plot(color='purple').show()

def enneper_transport(c_1, c_2, trange, num_to_plot=30):
    t = var('t')
    t_0 = trange[0]
    t_f = trange[1]

    u = var('u')
    v = var('v')
    X(u, v) = (enneper.equation[0], enneper.equation[1], enneper.equation[2])
    # X(u, v) = (-1/9*(u^2 - 3*v^2 - 3)*u, -1/9*(3*u^2 - v^2 + 3)*v, 1/3*u^2 - 1/3*v^2)
    print(X)

    X_u = diff(X, u)
    X_v = diff(X, v)

    P = PT(X, u, v, c_1, c_2, (t, t_0, t_f), step=0.01, initial_vec_length=1)

    X_plot = parametric_plot3d(X(u, v), (u, -3, 3), (v, -3, 3), frame=False,axes=False,color='purple',opacity=0.5)
    C_plot = parametric_plot3d(X(c_1(t), c_2(t)), (t, t_0, t_f), frame=False, Axes=False,color='blue',opacity=1,thickness=3)

    num_points = len(P)
    period = int(num_points / num_to_plot)
    vectors = []
    for ti,a,b in P[::period]:
        vec = (scale(a, X_u(c_1(ti), c_2(ti))) + scale(b, X_v(c_1(ti), c_2(ti))))
        vec = scale(0.5, vec)
        vectors.append(arrow(X(c_1(ti), c_2(ti)), X(c_1(ti), c_2(ti)) + vec))

    output = X_plot + sum(vectors) + C_plot
    output.show()

In [ ]:
c_1(t) = 1.5
c_2(t) = t
enneper_transport(c_1, c_2, (-3, 3))

In [ ]:
c_1(t) = t^3 + 2
c_2(t) = t^2
enneper_transport(c_1, c_2, (-1, 1))

In [ ]:
c_1(t) = t
c_2(t) = t^2
enneper_transport(c_1, c_2, (-sqrt(3), sqrt(3)))

In [ ]:
c_1(t) = sin(t)
c_2(t) = cos(t)
enneper_transport(c_1, c_2, (0, 2*pi))

In [ ]:
c_1(t) = 3*sin(t)
c_2(t) = 3*cos(t)
enneper_transport(c_1, c_2, (0, 2*pi), num_to_plot=120)

In [ ]:
c_1(t) = sin(t)
c_2(t) = t
enneper_transport(c_1, c_2, (-3, 3))

In [ ]:
x(z, x_0, y_0) = y_0 * cosh((z - x_0)/y_0)
X(theta, z, x_0, y_0) = (cos(theta) * x(z, x_0, y_0), sin(theta) * x(z, x_0, y_0), z)

In [ ]:
def get_surface(X, x_0, y_0, max_r):
    def condition(x, y, z):
        return bool(sqrt(x^2 + y^2) <= max_r)

    max_z = solve([sqrt(X(0, z, x_0, y_0)[0]^2 + X(0, z, x_0, y_0)[1]^2) == max_r], z)
    print(max_z)

    P = parametric_plot3d(X(theta, z, x_0, y_0), (theta, 0, 2*pi), (z, -z.subs(max_z), z.subs(max_z)))
    R = P.add_condition(condition)

    return R

In [ ]:
max_r = 8
y_0 = 1
get_surface(X, 0, y_0, max_r)

In [ ]:
y_0 = 1.5
get_surface(X, 0, y_0, max_r)

In [ ]:
y_0 = 2
get_surface(X, 0, y_0, max_r)

In [ ]:
y_0 = 2.5
get_surface(X, 0, y_0, max_r)

In [ ]:
y_0 = 3
get_surface(X, 0, y_0, max_r)